In [3]:
import pandas as pd
!pip install wikipedia
import wikipedia as wp
import numpy as np
import requests
import io
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

import folium

from geopy.geocoders import Nominatim

# Start by creating a new Notebook for this assignment.
# setup import

# Get the table from wiki page
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")

# Determine the index of your table
df = pd.read_html(html, header = 0)[0]

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
df.head()

# Merge rows with the same Post code
df = df.groupby(["Postal Code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df.head()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
df
df.shape

# Using the csv file for location coordinates
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

# Read the csv data
df_cord = pd.read_csv('GeoCord.csv') 
df_cord.head()

# Create Latitude and Longitude columns in df
df['Latitude'] = np.nan
df['Longitude'] = np.nan

# For each postcode in df, find corresponding coordinates in df_cord and assign it to df
for idx in df.index:
    cord_idx = df_cord['Postal Code'] == df.loc[idx, 'Postal Code']
    df.at[idx, 'Latitude'] = df_cord.loc[cord_idx, 'Latitude'].values
    df.at[idx, 'Longitude'] = df_cord.loc[cord_idx, 'Longitude'].values

# Display the results
df.head(20)


--2020-05-12 00:48:18--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.48.113.194, 158.85.108.86, 158.85.108.83
Connecting to cocl.us (cocl.us)|169.48.113.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2020-05-12 00:48:18--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-12 00:48:19--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-12 00:48:19--  https://ibm.box.com/p

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [4]:
toronto_df_new = df

address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [5]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

# create a new DataFrame with only boroughs that contain the word Toronto
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [6]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [7]:
CLIENT_ID = 'EIP0KY50ZNVCTNY0POLM3RY0UX2XADECAGSLJ0BS4DFVQ1V0' # your Foursquare ID
CLIENT_SECRET = 'KPY4KKCCBKMI450P4AEGOEMYXC1UQRO2KMK3MGGBBOLT53JG'  # your Foursquare Secret
VERSION = '20200401' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EIP0KY50ZNVCTNY0POLM3RY0UX2XADECAGSLJ0BS4DFVQ1V0
CLIENT_SECRET:KPY4KKCCBKMI450P4AEGOEMYXC1UQRO2KMK3MGGBBOLT53JG


In [12]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Postal Code'], toronto_df_new['Borough'], 
                                                  toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=EIP0KY50ZNVCTNY0POLM3RY0UX2XADECAGSLJ0BS4DFVQ1V0&client_secret=KPY4KKCCBKMI450P4AEGOEMYXC1UQRO2KMK3MGGBBOLT53JG&v=20200401 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
    

In [13]:
#Convert venues list to new dataframe

venues_df = pd.DataFrame(venues)


venues_df.columns = ['Postal Code', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()


(1638, 9)


,Postal Code,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Textile Museum of Canada,43.654396,-79.386500,Art Museum
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Cafe Plenty,43.654571,-79.389450,Café
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Japango,43.655268,-79.385165,Sushi Restaurant


In [14]:
venues_df.groupby(["Postal Code", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Postal Code,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,42,42,42,42,42,42
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"India Bazaar, The Beaches West",42,42,42,42,42,42
M4M,East Toronto,Studio District,42,42,42,42,42,42
M4N,Central Toronto,Lawrence Park,42,42,42,42,42,42
M4P,Central Toronto,Davisville North,42,42,42,42,42,42
M4R,Central Toronto,North Toronto West,42,42,42,42,42,42
M4S,Central Toronto,Davisville,42,42,42,42,42,42
M4T,Central Toronto,"Moore Park, Summerhill East",42,42,42,42,42,42


In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['Postal Code'] = venues_df['Postal Code'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1638, 35)


,Postal Code,Borough,Neighborhoods,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Park,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [16]:
toronto_grouped = toronto_onehot.groupby(["Postal Code", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped


(39, 35)


,Postal Code,Borough,Neighborhoods,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Park,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.02381
1,M4K,East Toronto,"The Danforth West, Riverdale",0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.02381
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.02381
3,M4M,East Toronto,Studio District,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.02381
4,M4N,Central Toronto,Lawrence Park,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.02381
5,M4P,Central Toronto,Davisville North,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.02381
6,M4R,Central Toronto,North Toronto West,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.02381
7,M4S,Central Toronto,Davisville,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.02381
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.02381
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.02381


In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Postal Code', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,Postal Code,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
3,M4M,East Toronto,Studio District,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
4,M4N,Central Toronto,Lawrence Park,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
5,M4P,Central Toronto,Davisville North,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
6,M4R,Central Toronto,North Toronto West,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
7,M4S,Central Toronto,Davisville,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
8,M4T,Central Toronto,"Moore Park, Summerhill East",Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


In [21]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["Postal Code", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [23]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("Postal Code"), on="Postal Code")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!


(39, 16)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


In [24]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
23,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
25,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
27,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
20,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
28,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


In [26]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
21,Downtown Toronto,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
22,Central Toronto,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
23,Central Toronto,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
24,Central Toronto,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
25,Downtown Toronto,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
26,Downtown Toronto,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
27,Downtown Toronto,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
20,Downtown Toronto,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
28,Downtown Toronto,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [31]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


## Most of the neighborhoods fall into Cluster 1 which are the areas with cafe, restaurants, supermarkets etc